In [2]:
with open('23a.txt') as f:
    lines = f.readlines()

In [3]:
from typing import List, Tuple

In [4]:
points_dict = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

In [11]:
A_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,3),(3,3)]
B_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,5),(3,5)]
C_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,7),(3,7)]
D_spots = [(1,1),(1,2),(1,4),(1,6),(1,8),(1,10),(1,11),(2,9),(3,9)]

In [5]:
def base_grid():
    grid = [['.']*13 for i in range(5)]
    grid[0] = ['#'] * 13
    grid[1][0] = '#'
    grid[1][12] = '#'
    grid[2][0:3] = '#' * 3
    grid[2][10:13] = '#' * 3
    grid[2][4] = '#'
    grid[2][6] = '#'
    grid[2][8] = '#'
    grid[3][2] = '#'
    grid[3][4] = '#'
    grid[3][6] = '#'
    grid[3][8] = '#'
    grid[3][10] = '#'
    grid[4][2:11] = '#' * 9
    return grid

In [6]:
class Game_State:
    
    def __init__(self, pods: List[Tuple[int,int,chr]]):
        self.pods = pods
        self.base = base_grid()
        for coords in self.pods:
            row,col,ch = coords
            self.base[row][col] = ch

    def check_point(self, row: int, col: int) -> bool:
        if self.base[row][col] != '.':
            return False
         
    def __str__(self):
        string = ''
        for i in range(len(self.base)):
            string += str(self.base[i])
            if i != len(self.base) - 1: string += '\n'
        return string
    


In [7]:
init_pods = []
for row in range(len(lines)):
    for col in range(len(lines[row])):
        if lines[row][col] == 'A':
            init_pods.append((row,col,'A'))
        elif lines[row][col] == 'B':
            init_pods.append((row,col,'B'))
        elif lines[row][col] == 'C':
            init_pods.append((row,col,'C'))
        elif lines[row][col] == 'D':
            init_pods.append((row,col,'D'))
init_state = Game_State(init_pods)

In [8]:
print(init_state)

['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#']
['#', '#', '#', 'D', '#', 'A', '#', 'D', '#', 'C', '#', '#', '#']
['.', '.', '#', 'C', '#', 'A', '#', 'B', '#', 'B', '#', '.', '.']
['.', '.', '#', '#', '#', '#', '#', '#', '#', '#', '#', '.', '.']


In [ ]:
def valid_end(goal: Tuple[int, int], flavour: chr, hallway: bool, grid: Game_State) -> bool:
        if flavour == 'A' and (row, col) not in A_spots:
            return False
        elif flavour == 'B' and (row, col) not in B_spots:
            return False
        elif flavour == 'C' and (row, col) not in C_spots:
            return False
        elif flavour == 'D' and (row, col) not in D_spots:
            return False
        

In [10]:
def valid_path(start: Tuple[int, int], goal: Tuple[int, int], grid: Game_State) -> bool:
    # check if have to move up first
    if goal[0] > start[0]:
        for i in range(goal[0] - start[0] + 1):
            if not grid.check_point(start[0] + i, start[1]):
                return False

    # check moving right
    if goal[1] > start[1]:
        for i in range(goal[1] - start[1] + 1):
            if not grid.check_point(goal[0], start[1] + 1):
                return False
    
    #check moving left
    if goal[1] < start[1]:
        

IndentationError: expected an indented block (Temp/ipykernel_10156/2311239632.py, line 6)

In [ ]:
def find_pods_moves(gamestate: Game_State, pod: Tuple[int, int, int]):
    moves = []
    if not pod in gamestate.pods:
        print("pod not actually in game state!")
        return None
    if gamestate.base[pod[0]+1][pod[1]] == '.':
        new_pods = gamestate.pods.copy()
        new_pods.remove(pod)
        new_pods.append((pod[0]+1, pod[1], pod[2]))



SyntaxError: invalid syntax (2022314871.py, line 5)

In [ ]:
def find_possible_moves(gamestate: Game_State):
    for pod in gamestate.pods:
        

SyntaxError: expected ':' (1391122201.py, line 1)

In [ ]:
# given a state, recursively check all possible states. keep a list of states in memory to avoid loops.
def move(pods: List[Tuple[int, int]]):
    

IndentationError: expected an indented block after function definition on line 2 (20354518.py, line 3)